In [8]:
import warnings
warnings.filterwarnings("ignore")
import argparse
import os
import time
from Network import BrainNet
from Loss import *
from NeuralODE import *
from Utils import *
from Registration import registration, evaluation_anova

In [9]:
parser = argparse.ArgumentParser()
# File path
parser.add_argument("--savepath", type=str,
                    dest="savepath", default='./result',
                    help="path for saving results")
parser.add_argument("--fixed", type=str,
                    dest="fixed", default='./data/OAS1_0001_MR1/brain.nii.gz',
                    help="fixed image data path")
parser.add_argument("--moving", type=str,
                    dest="moving", default='./data/OAS1_0002_MR1/brain.nii.gz',
                    help="moving image data path")
parser.add_argument("--fixed_seg", type=str,
                    dest="fixed_seg", default='./data/OAS1_0001_MR1/brain_aseg.nii.gz',
                    help="fixed image segmentation data path")
parser.add_argument("--moving_seg", type=str,
                    dest="moving_seg", default='./data/OAS1_0002_MR1/brain_aseg.nii.gz',
                    help="moving image segmentation data path")
# Model configuration
parser.add_argument("--ds", type=int,
                    dest="ds", default=2,
                    help="specify output downsample times.")
parser.add_argument("--bs", type=int,
                    dest="bs", default=16,
                    help="bottleneck size.")
parser.add_argument("--smoothing_kernel", type=str,
                    dest="smoothing_kernel", default='AK',
                    help="AK: Averaging kernel; GK: Gaussian Kernel")
parser.add_argument("--smoothing_win", type=int,
                    dest="smoothing_win", default=15,
                    help="Smoothing Kernel size")
parser.add_argument("--smoothing_pass", type=int,
                    dest="smoothing_pass", default=1,
                    help="Number of Smoothing pass")
# Training configuration
parser.add_argument("--time_steps", type=int,
                    dest="time_steps", default=2,
                    help="number of time steps between the two images, >=2.")
parser.add_argument("--optimizer", type=str,
                    dest="optimizer", default='Euler',
                    help="Euler or RK.")
parser.add_argument("--STEP_SIZE", type=float,
                    dest="STEP_SIZE", default=0.001,
                    help="step size for numerical integration.")
parser.add_argument("--epoches", type=int,
                    dest="epoches", default=300,
                    help="No. of epochs to train.")
parser.add_argument("--NCC_win", type=int,
                    dest="NCC_win", default=21,
                    help="NCC window size")
parser.add_argument("--lr", type=float,
                    dest="lr", default=0.005,
                    help="Learning rate.")
parser.add_argument("--lambda_J", type=int,
                    dest="lambda_J", default=2.5,
                    help="Loss weight for neg J")
parser.add_argument("--lambda_df", type=int,
                    dest="lambda_df", default=0.05,
                    help="Loss weight for dphi/dx")
parser.add_argument("--lambda_v", type=int,
                    dest="lambda_v", default=0.00005,
                    help="Loss weight for neg J")
parser.add_argument("--loss_sim", type=str,
                    dest="loss_sim", default='NCC',
                    help="Similarity measurement")
# Debug
parser.add_argument("--debug", type=bool,
                    dest="debug", default=False,
                    help="debug mode")
# Device
parser.add_argument("--device", type=str,
                    dest="device", default='cuda:0',
                    help="gpu: cuda:0; cpu: cpu")

parser.add_argument("--f", type=str,
                    dest='not_important')

config, unknown = parser.parse_known_args()
if not os.path.isdir(config.savepath):
    os.makedirs(config.savepath)

In [11]:
dice_data = {
    'epochs':[],
    'learning_rate':[],
    'smoothing_kernel':[],
    'file':[],
    'value':[]
}

jacob_data = {
    'epochs':[],
    'learning_rate':[],
    'smoothing_kernel':[],
    'file':[],
    'value':[]
}

epoches = [50,100,200]
learning_rates = [0.001, 0.005, 0.01]
kernel_sizes = [5,10,15]
files = [
            (
                'data (1)/OASIS_OAS1_0001_MR1/aligned_norm.nii.gz',
                'data (1)/OASIS_OAS1_0002_MR1/aligned_norm.nii.gz',
                'data (1)/OASIS_OAS1_0001_MR1/aligned_seg35.nii.gz',
                'data (1)/OASIS_OAS1_0002_MR1/aligned_seg35.nii.gz'
            ),
            (
                'data (1)/OASIS_OAS1_0002_MR1/aligned_norm.nii.gz',
                'data (1)/OASIS_OAS1_0003_MR1/aligned_norm.nii.gz',
                'data (1)/OASIS_OAS1_0002_MR1/aligned_seg35.nii.gz',
                'data (1)/OASIS_OAS1_0003_MR1/aligned_seg35.nii.gz'
            )
      ]


#config.devid=ce

for kernel in kernel_sizes:
    config.smoothing_win = kernel
    for lr in learning_rates:
        config.lr = lr
        for epoch in epoches:
            config.epoches = epoch
            for fixed, moving, fixed_seg, moving_seg in files:
                device = torch.device(config.device)
                fixed = load_nii(config.fixed)
                moving = load_nii(config.moving)
                assert fixed.shape == moving.shape  # two images to be registered must in the same size
                df, df_with_grid, warped_moving = registration(config, device, moving, fixed)
                mean_dice, std_dice, ratio_neg_J = evaluation_anova(config, device, df, df_with_grid)

                print(f'Done with running:\n\tkernel: {kernel}\n\tlearning_rate: {lr}\n\tepoches: {epoch}')

                dice_data['epochs'].append(epoch)
                dice_data['learning_rate'].append(lr)
                dice_data['smoothing_kernel'].append(kernel)
                dice_data['file'].append(f'fixed: {fixed}, moving: {moving}')
                dice_data['value'].append(mean_dice)

                jacob_data['epochs'].append(epoch)
                jacob_data['learning_rate'].append(lr)
                jacob_data['smoothing_kernel'].append(kernel)
                jacob_data['file'].append(f'fixed: {fixed}, moving: {moving}')
                jacob_data['value'].append(ratio_neg_J)



KeyboardInterrupt: 